In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
from surgeNN.models import build_LSTM_per_variable, build_LSTM_stacked, build_LSTM_stacked_with_attention, build_ConvLSTM2D_with_channels
from surgeNN.io import load_predictand,load_predictors
from surgeNN.utils import plot_loss_evolution, get_train_test_val_idx, normalize_timeseries, rmse, sequenced_dataset_from_dataset
from scipy import stats
from sklearn.metrics import confusion_matrix
from target_relevance import TargetRelevance #first, in terminal, do->'mamba install kdepy'
from tqdm import tqdm

2024-05-29 15:05:13.702706: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Load in & preprocess data

In [2]:
tg = 'den_helder-denhdr-nld-rws.csv' #site to predict
predictand = load_predictand('/home/jovyan/test_surge_models/input/t_tide_6h_anoms_deseasoned_predictands',tg) #open predictand csv
#predictand = load_predictand('/Users/timhermans/Documents/Github/surgeNN/input/predictands_6hourly',tg) #open predictand csv
predictors = load_predictors('/home/jovyan/test_surge_models/input/predictors_6hourly',tg,5) #open predictor xarray dataset
predictors = predictors.sel(time=slice('1980','2016')) #period for which we have hydrodynamic output as well

predictor_timesteps = predictors.time.to_dataframe() #store predictor timesteps in separate variable for later

# only use predictands at timesteps for which we have predictor values:
predictand = predictand[(predictand['date']>=predictors.time.isel(time=0).values) & (predictand['date']<=predictors.time.isel(time=-1).values)] 

#select predictors at predictand timesteps
predictors = predictors.sel(time=predictand['date'].values)
predictors = (predictors-predictors.mean(dim='time'))/predictors.std(dim='time',ddof=0) #normalize each variable in dataset
predictand['surge'] = predictand['surge'].rolling(window=5,min_periods=1,center=True).mean()
predictand['surge'] = normalize_timeseries(predictand['surge']) #normalize predictands

surge_obs = predictand['surge'].values #get values from dictionary

my_predictors = np.stack((predictors.msl.values,predictors.w.values,predictors.u10.values,predictors.v10.values),axis=-1)
#my_predictors = np.reshape(my_predictors,(len(predictors.time),len(predictors.latitude) * len(predictors.longitude) * 4)) #for LSTM with stacked input

idx_train,idx_test,idx_val = get_train_test_val_idx(my_predictors,surge_obs,[.6,.2,.2],shuffle=False) #split data into training, validation and testing and get indices

Create parameter space:

In [3]:

n_steps = np.arange(4,10,2).astype('int')
batch_size = np.array([64,128]).astype('int')
alpha = np.arange(2,5,0.5)
n_lstm = np.array([1,2]).astype('int')
n_dense = np.array([1,2,3]).astype('int')
n_units = np.array([24,48,72]).astype('int')
dropout = np.array([0.1,0.2,0.3])
lrs = np.array([5e-4,1e-5,5e-5, 1e-6])
l1s = np.array([0.01,0.02,0.03])
'''
n_steps = np.arange(8,14,2).astype('int')
batch_size = np.array([64,128]).astype('int')
alpha = np.arange(3.5,6,0.5)
n_lstm = np.array([1,2]).astype('int')
n_dense = np.array([1,2]).astype('int')
n_units = np.array([24,48]).astype('int')
dropout = np.array([0.1,0.2])
lrs = np.array([1e-5,3e-5,5e-5])
l1s = np.array([0.02])
'''
possible_settings = [n_steps,batch_size,alpha,n_lstm,n_dense,n_units,dropout,lrs,l1s]


In [5]:
settings = []
rs = []
rmses = []
fps = []
fns = []
tps = []
tns = []
for i in tqdm(np.arange(20)):
    i_settings = [np.random.choice(k) for k in possible_settings]
    settings.append(i_settings)
    
    n_steps = i_settings[0]
    batch_size = i_settings[1]
    
    #generate loss weights:
    target_relevance = TargetRelevance(surge_obs[np.append(idx_train,idx_val)], alpha=i_settings[2])
    weights = target_relevance.eval(surge_obs[np.append(idx_train,idx_val)]).flatten()
    
    #generate input datasets:
    xx_train = sequenced_dataset_from_dataset(tf.data.Dataset.from_tensor_slices(my_predictors[idx_train,...]), n_steps, batch_size) #if passing variables together as channels
    yy_train = sequenced_dataset_from_dataset(tf.data.Dataset.from_tensor_slices(surge_obs[idx_train][n_steps-1::]), 1, batch_size)
    ww_train = sequenced_dataset_from_dataset(tf.data.Dataset.from_tensor_slices(weights[idx_train][n_steps-1::]), 1, batch_size)
    zz_train = tf.data.Dataset.zip((xx_train,yy_train,ww_train))

    xx_val = sequenced_dataset_from_dataset(tf.data.Dataset.from_tensor_slices(my_predictors[idx_val,...]), n_steps, batch_size) #if passing variables together as channels
    yy_val = sequenced_dataset_from_dataset(tf.data.Dataset.from_tensor_slices(surge_obs[idx_val][n_steps-1::]), 1, batch_size)
    ww_val = sequenced_dataset_from_dataset(tf.data.Dataset.from_tensor_slices(weights[idx_val][n_steps-1::]), 1, batch_size)
    zz_val = tf.data.Dataset.zip((xx_val,yy_val,ww_val))
    
    xx_test = sequenced_dataset_from_dataset(tf.data.Dataset.from_tensor_slices(my_predictors[idx_test,...]), n_steps, batch_size)

    #build model:
    #model = build_LSTM_stacked(i_settings[3], i_settings[4], 
    #                          (np.ones(i_settings[3])*i_settings[5]).astype(int), 
    #                          (np.ones(i_settings[4])*i_settings[5]).astype(int), n_steps,20,20,4, 'model0',
    #                           i_settings[6], i_settings[7], 'mae',l2=i_settings[8])
    model = build_ConvLSTM2D_with_channels(i_settings[3], i_settings[4],  (np.ones(i_settings[3])*i_settings[5]).astype(int), (np.ones(i_settings[4])*i_settings[5]).astype(int), n_steps,20,20, 4, 'model0',i_settings[6], i_settings[7], 'mae',l2=i_settings[8])
    
    #train model:
    history = model.fit(zz_train,epochs=40,validation_data=zz_val,callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5,
                        restore_best_weights=True)],verbose=0)
    
    
    
    #infer test set and evaluate extremes
    surge_cnn_test = model.predict(xx_test,verbose=0).flatten()
   
    threshold_pct = 98 #percentile of storm surge data to look at

    surge_obs_test = surge_obs[idx_test][n_steps-1::] #surge_obs[idx_test]
    threshold_value = np.percentile(surge_obs[idx_test][n_steps-1::],threshold_pct) #threshold value

    surge_obs_test_exceedances = (surge_obs[idx_test][n_steps-1::]>=threshold_value).flatten() #find where storm surges exceed threshold (extremes), for observations
    surge_cnn_test_exceedances = (surge_cnn_test>=threshold_value).flatten() #find where storm surges exceed threshold (extremes), for predictions with CNN

    rs.append(np.corrcoef(surge_cnn_test[surge_obs_test_exceedances],surge_obs_test[surge_obs_test_exceedances])[0][1])
    rmses.append(rmse(surge_cnn_test[surge_obs_test_exceedances],surge_obs_test[surge_obs_test_exceedances]))

    cmat = confusion_matrix(surge_obs_test_exceedances,surge_cnn_test_exceedances)
    tns.append(cmat[0,0])
    fns.append(cmat[1,0])
    fps.append(cmat[0,1])
    tps.append(cmat[1,1])


  0%|          | 0/50 [00:00<?, ?it/s]

TargetRelevance alpha: 3.5
Using Silverman Bandwidth 0.12847424667144736
Epoch 1/40


2024-05-29 15:06:48.996280: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel0/spatial_dropout2d_1/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


247/247 - 37s - loss: 1.2475 - accuracy: 0.0000e+00 - val_loss: 2.2631 - val_accuracy: 0.0000e+00 - 37s/epoch - 148ms/step
Epoch 2/40


  0%|          | 0/50 [00:48<?, ?it/s]


KeyboardInterrupt: 

In [5]:
ds = xr.Dataset(
    data_vars=dict(
        corr=(["i"], rs),
        rmse=(["i"], rmses),
        tps=(["i"], tps),
        tns=(["i"], tns),
        fps=(["i"], fps),
        fns=(["i"], fns),
    ),
    coords=dict(
        settings=(["i",'setting'], settings),
    ),
)
ds['settings'] = ['n_steps','batch_size','alpha','n_lstm','n_dense','n_units','dropout','lrs','l1s']
ds.to_netcdf('conv2dlstm_tuning_output_6h.nc',mode='w')

In [6]:
ds

<xarray.Dataset>
Dimensions:   (i: 100, settings: 9)
Coordinates:
  * settings  (settings) <U10 'n_steps' 'batch_size' 'alpha' ... 'lrs' 'l1s'
Dimensions without coordinates: i
Data variables:
    corr      (i) float64 0.8045 0.8288 0.6968 0.7025 ... 0.8058 0.7525 0.7384
    rmse      (i) float64 0.6295 0.7433 0.8663 0.7776 ... 0.6405 0.7423 0.746
    tps       (i) int64 331 259 264 292 304 329 337 ... 312 310 331 330 279 326
    tns       (i) int64 20187 20513 20501 20346 ... 20136 20403 20472 20367
    fps       (i) int64 422 94 106 267 189 408 265 ... 328 231 473 204 137 240
    fns       (i) int64 90 162 157 129 117 92 84 77 ... 93 109 111 90 91 142 95

In [13]:
np.sort(rs)

array([0.36812176, 0.50603313, 0.51512277, 0.52026681, 0.52333824,
       0.53281848, 0.54393787, 0.55858491, 0.58329246, 0.59841041,
       0.59866841, 0.60002311, 0.61359457, 0.61383089, 0.61398428,
       0.61600893, 0.62204333, 0.62435331, 0.62658238, 0.62733723,
       0.62985157, 0.63067465, 0.63104616, 0.63519119, 0.6429531 ,
       0.64479494, 0.645641  , 0.64912904, 0.64965616, 0.65326881,
       0.65353228, 0.65486472, 0.6551271 , 0.65941609, 0.65979869,
       0.6598983 , 0.65989904, 0.66055651, 0.66411969, 0.66506543,
       0.66567462, 0.66584121, 0.66654676, 0.66716217, 0.67072511,
       0.67285183, 0.67330577, 0.67792391, 0.6783692 , 0.68052458,
       0.68162014, 0.68218334, 0.68247329, 0.68342966, 0.68609478,
       0.69054637, 0.69229629, 0.69339751, 0.69597832, 0.69629711,
       0.69639434, 0.69722307, 0.69877936, 0.70106644, 0.70159904,
       0.70223663, 0.70264137, 0.70507679, 0.70522206, 0.70594974,
       0.70602835, 0.70822312, 0.70959484, 0.71009856, 0.71187

In [9]:
np.sort(rmses)

array([0.59215893, 0.62117068, 0.642189  , 0.65420242, 0.66397179,
       0.66670912, 0.70029141, 0.70390991, 0.70940051, 0.71883975,
       0.73353879, 0.74600366, 0.75232987, 0.76306273, 0.76805886,
       0.77471956, 0.77821339, 0.77849542, 0.77859503, 0.77878076,
       0.77969483, 0.78082952, 0.78232195, 0.7864492 , 0.78856918,
       0.78941644, 0.79061478, 0.79851892, 0.80258214, 0.80898304,
       0.81600016, 0.81726203, 0.82068551, 0.82246407, 0.82822891,
       0.82904851, 0.83217327, 0.83331532, 0.83444531, 0.83483015,
       0.83642149, 0.8367848 , 0.83752544, 0.8422461 , 0.85442052,
       0.85528837, 0.85559634, 0.85749802, 0.85831919, 0.85932405,
       0.85989078, 0.87085564, 0.87207151, 0.87438737, 0.87519432,
       0.87530865, 0.87834771, 0.88167988, 0.88246409, 0.88517785,
       0.88847299, 0.89057254, 0.89209484, 0.8976012 , 0.90011072,
       0.90228963, 0.91998259, 0.92178453, 0.93253527, 0.93828928,
       0.93858717, 0.93918392, 0.93963751, 0.94041513, 0.94573

In [10]:
np.array(settings)[np.argsort(rmses),:][0:5]

array([[6.00e+00, 6.40e+01, 4.50e+00, 1.00e+00, 3.00e+00, 7.20e+01,
        2.00e-01, 1.00e-05, 2.00e-02],
       [4.00e+00, 6.40e+01, 5.50e+00, 2.00e+00, 1.00e+00, 2.40e+01,
        1.00e-01, 1.00e-05, 2.00e-02],
       [6.00e+00, 1.28e+02, 5.50e+00, 2.00e+00, 3.00e+00, 4.80e+01,
        2.00e-01, 1.00e-05, 3.00e-02],
       [4.00e+00, 6.40e+01, 3.50e+00, 2.00e+00, 2.00e+00, 7.20e+01,
        1.00e-01, 5.00e-05, 1.00e-02],
       [2.00e+00, 1.28e+02, 4.50e+00, 2.00e+00, 1.00e+00, 2.40e+01,
        1.00e-01, 5.00e-05, 1.00e-02]])

In [12]:
i_settings

[2, 64, 1.0, 1, 3, 24, 0.1, 5e-05, 0.01]